In [4]:
import pymysql,json,datetime
import yaml
from pathlib import Path
from flask import Flask, url_for,render_template, request,session, redirect,send_from_directory,make_response 
from flask_session import Session
from datetime import timedelta
from user_ckn import user,baseObject
import time,json,requests,pymysql
import datetime
df = yaml.safe_load(Path('config.yml').read_text())
df
from flask import Flask, url_for,render_template, request,session, redirect,send_from_directory,make_response 
from flask_session import Session
from datetime import timedelta
from user_ckn import user,baseObject
import time,json,requests,pymysql
import datetime
from user_ckn import user,baseObject
import pymysql
import yfinance as yf
import plotly.graph_objects as go
from flask import Flask, render_template

### Login Check

In [ ]:
def login_check(det):
    error = []
    conn = pymysql.connect(host='mysql.clarksonmsda.org', port=3306, user='gonuguc',
                           passwd='Chandu@1604', db='gonuguc_CryptoKoiN', autocommit=True)
    cur = conn.cursor(pymysql.cursors.DictCursor)

    sql = "SELECT * FROM `User_data` WHERE username = %s"
    cur.execute(sql, (det[0],))
    data = cur.fetchall()

    if len(data) == 0:
        return ["Incorrect Username"]

    if data[0]["password_hash"] != det[1]:
        error.append("Wrong password")
        
    if len(error) == 0:
        i_data = [data[0]["user_id"]]
        sql = "INSERT INTO `User_Activity` (user_id) VALUES (%s)"
        cur.execute(sql, i_data)
        return "Successfully Logged In"
    else:
        return error


### Forgot password

In [17]:
def reset_password(det):
    import pymysql
    import datetime

    data = []
    error = []

    conn = pymysql.connect(
        host='mysql.clarksonmsda.org',
        port=3306,
        user='gonuguc',
        passwd='Chandu@1604',
        db='gonuguc_CryptoKoiN',
        autocommit=True
    )
    cur = conn.cursor(pymysql.cursors.DictCursor)

    # Fetch user data by username
    sql = '''SELECT * 
             FROM `User_data` 
             WHERE username = %s'''
    cur.execute(sql, (det[0],))
    data = cur.fetchall()
    print(data[0]['date_of_birth'])
    print(type(data[0]['date_of_birth']))

    if len(data) == 0:
        return 'Incorrect Username'

    # Perform validation checks
    if data[0]['email'] != det[1]:
        error.append('Incorrect Email')

    if data[0]['date_of_birth'] != det[2]:
        error.append('Incorrect DOB')

    if det[3] != det[4]:
        error.append('Password didn\'t match')

    if len(det[3]) < 8:
        error.append('Password should contain a minimum of 8 characters')

    # Return based on error validation
    if len(error) == 0:
        # Update password using a parameterized query
        sql = '''UPDATE `User_data`
                 SET password_hash = %s
                 WHERE user_id = %s'''
        cur.execute(sql, (det[3], data[0]['user_id']))
        conn.commit()
        return 'Password updated successfully'
    else:
        return error
    
    
det = ['chandu','gonuguc@clarkson.edu',datetime.datetime.strptime('2000-04-16','%Y-%m-%d').date(),'chandu@16','chandu@16']
reset_password(det)


2000-04-16
<class 'datetime.date'>


'Password updated successfully'

In [9]:
import datetime

date_object = datetime.datetime.strptime('1999-04-14', '%Y-%m-%d')

formatted_date = date_object.strftime('%Y-%m-%d')

print(formatted_date)  # Output: 1999-04-14


1999-04-14


### Sign Up

In [113]:
#username, email, dob, pwd, repwd
def signup_check(det):
    data = []
    error = []
    conn = pymysql.connect(host='mysql.clarksonmsda.org', port=3306, user='gonuguc',
                        passwd='Chandu@1604', db='gonuguc_CryptoKoiN', autocommit=True)
    cur = conn.cursor(pymysql.cursors.DictCursor)
    
    fname = det[0]
    lname = det[1]
    uname = det[2]
    email = det[3]
    dob = det[4]
    pwd = det[5]
    rt_pwd = det[6]
    role = det[7]

    sql = '''SELECT * 
            FROM `User_data` 
            WHERE username = %s OR email = %s'''
    cur.execute(sql, (uname, email))

    data = cur.fetchall()

    if len(data) != 0:
        if data[0]['username'] == uname:
            return 'Username found, try another'
        elif data[0]['email'] == email:
            return 'Email found, sign up with another email or log in with this email'

    if pwd != rt_pwd:
        return 'Password doesn\'t match'
    
    if len(pwd) < 8:
        error.append('Password should contain a minimum of 8 characters')
    
    else:
        i_data = [fname,lname,uname,email,pwd,role] 
        sql = '''INSERT INTO `User_data`(firstname,lastname,username, email, password_hash, role)
        VALUES (%s,%s,%s,%s,%s,%s )'''
        cur.execute(sql,i_data)
        return 'successfully created a account'


In [3]:
ckn = [{'asset_id': 22, 'quantity': 23.0, 'acquisition_date': '2024-11-28 17:11:09', 'user_id': 3, 'crypto_id': 21},
{'asset_id': 30, 'quantity': 1.0, 'acquisition_date': '2024-11-28 17:11:09', 'user_id': 3, 'crypto_id': 8}]

In [ ]:
shares = 3
trade_type = 'trail'
coin_price = 3
user_id = 3
crypto_id = 3
final_amount = 9
tax = 1.5

u = user()
u.tn = 'Transaction_History'
u.fields = []
u.getFields()
print(u.fields)
print(u.pk)
u.createBlank()
u.data
u.data[0] = dict(zip(u.fields,[ trade_type,shares, coin_price, datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), user_id, crypto_id,final_amount,tax]))
u.insert()

In [ ]:

# Coin Details Route
@app.route('/details/<coin_id>')
def coin_details(coin_id):
    u = user()
    u.cur.execute("SELECT * FROM `Crypto_coins` WHERE coin_symbol = %s", (coin_id,))
    coin = u.cur.fetchone()

    if not coin:
        return render_template('details.html', message="Coin not found"), 404
    
    # Fetch candlestick data using yfinance
    try:
        ticker = yf.Ticker(f"{coin['coin_symbol'].upper()}-USD")
        data = ticker.history(period="1y")

        if data.empty:
            graph_html = "<p>No historical data available for this coin.</p>"
        else:
            # Generate candlestick chart
            fig = go.Figure(data=[go.Candlestick(
                    x=data.index,
                    open=data['Open'],
                    high=data['High'],
                    low=data['Low'],
                    close=data['Close']
            )])
            # Add SMA
            data['20_SMA'] = data['Close'].rolling(window=20).mean()
            data['50_SMA'] = data['Close'].rolling(window=50).mean()
            fig.add_trace(go.Scatter(x=data.index, y=data['20_SMA'], mode='lines', name='20 SMA'))
            fig.add_trace(go.Scatter(x=data.index, y=data['50_SMA'], mode='lines', name='50 SMA'))

            fig.update_layout(
                    title=f"<b>{coin['coin_name']} Candlestick Chart</b>",
                    yaxis_title="Price (USD)",
                    xaxis_title="Date",
                    autosize=True,
                    margin=dict(l=20, r=20, t=40, b=20),
                    width=800
                )
            graph_html = fig.to_html(full_html=False)

    except Exception as e:
        graph_html = f"<p>Error generating chart: {e}</p>"

    return render_template('details.html', graph_html=graph_html, coin=coin, navigation_items=True)


[]

In [ ]:
from user_ckn import user,baseObject
'''
u = user()
u.tryLogin('srikanth','123456')
u.data[0]
user_id = u.data[0]['user_id']
wallet_balance = u.data[0]['wallet_balance']
sql = """
            SELECT c.cryptocoin_id, c.coin_symbol, c.coin_name, c.current_price, 
                IFNULL(a.quantity, 0) AS shares_owned
            FROM Crypto_coins c
            LEFT JOIN User_Assets a ON c.cryptocoin_id = a.crypto_id AND a.user_id = %s
        """
u.cur.execute(sql,user_id)
coins = u.cur.fetchall()


coin_symbol = 'BTC'
u.tn = 'Crypto_coins'
u.getByField('coin_symbol',coin_symbol)
coin = u.data[0]
u.cur.execute("UPDATE User_data SET wallet_balance = %s WHERE user_id = %s",
                    (wallet_balance, user_id))
u.tn = 'User_data'
u.getById(user_id)
u.data[0]['wallet_balance'] = 150

u.tn = 'User_Assets'
u.fields = []
u.getFields()
print(u.fields)
print(u.pk)
u.getByField('user_id',user_id)
print(u.data[0])
u.data[0]['quantity'] = 23
u.data[0]['acquisition_date'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
u.data[0]['crypto_id'] = 21
u.insert()'''


u = user()
u.tn = 'User_Assets'
u.getByField('user_id',1)
quantity = 0
if len(u.data)>0:
    quantity = sum(row['quantity'] for row in u.data if row['crypto_id'] == 11)
quantity
u.data.clear()

u.tn = 'User_Assets'
u.fields = []
u.getFields()
print(u.fields)
print(u.pk)
u.createBlank()
print(u.data)
u.data[0] = dict(zip(u.fields,[25,datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),25,25]))
u.insert()

In [ ]:
u.data.clear()
u.tn = 'Transaction_History'
u.fields = []
u.getFields()
print(u.fields)
print(u.pk)
u.createBlank()
u.data
u.data[0] = dict(zip(u.fields,[trade_type,shares, coin_price, datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),user_id,crypto_id,final_amount,tax]))
u.insert()
u.data.clear()

In [ ]:
shares = 3
trade_type = 'trail'
coin_price = 3
user_id = 3
crypto_id = 3
final_amount = 9
tax = 1.5


u = user()
u.tn = 'Transaction_History'
u.fields = []
u.getFields()
print(u.fields)
print(u.pk)
u.createBlank()
u.data
u.data[0] = dict(zip(u.fields,[ trade_type,shares,   coin_price, datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), user_id, crypto_id,final_amount,tax]))
u.insert()

In [ ]:
u = user()

u.tn = 'Transaction_History'
u.fields = []
u.getFields()
print(u.fields)
print(u.pk)
print(u.createBlank())

'''u.data[0]['transaction_type'] = 'buy' #trade_type
u.data[0]['quantity'] = 2.3 #shares
u.data[0]['price_at_transaction'] = 45.6 #coin_price
u.data[0]['transaction_timestamp'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
u.data[0]['user_id'] = 64 #user_id
u.data[0]['crypto_id'] = 23 #crypto_id
u.data[0]['tax'] = 3 #tax
u.data[0]['total_amount'] = round(((2.3*45.6) +  3),2 ) #final_amount
'''
u.data[0] = dict(zip(u.data[0].keys(),['buy',2,2.3,datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),34,34,34,34]))
u.insert()


In [ ]:
#profile
user_id = 3
u= user()
u.getById(user_id) 
u_details = u.data[0]
u_details['username'] = u_details['username'].upper()
u_details['created_at'] = u_details['created_at'].strftime("%B %d, %Y") 
u_details

In [ ]:
# transactions
user_id = 3
u= user()
sql = '''SELECT cc.image_url,cc.coin_name,th.transaction_type,th.quantity,th.total_amount,th.transaction_timestamp
FROM `Transaction_History` th, Crypto_coins cc WHERE th.crypto_id = cc.cryptocoin_id AND th.user_id =%s ORDER BY `th`.`transaction_timestamp` DESC'''
u.cur.execute(sql,user_id)
th = u.cur.fetchall()

In [15]:
#wallet

user_id= 4
u = user()
u.getById(user_id)
u.data[0]['wallet_balance'] = 1075
u.update()